# FlatBean

This is a minimal implementation of the symbolic represenation feature in a pulse building library.

In [14]:
from copy import copy
from numpy import sin
from broadbean.segment import Segment, SegmentGroup
from broadbean.element import Element
from broadbean.sequence_simple import Sequence

Segments are defined with a function and list of parameters that can be strings=symbols or values.
Specific segments (Atoms) are not yet implemented, but they behave identically

In [15]:
s = Segment(function=lambda time,x: sin(time + x), duration='waiting_time', x='phase')
s2 = Segment(function=lambda time,x: sin(time + x), duration='pulse_time', x='phase')

The symbols get filled in through a context, defined by a dictionary at the latest possible time, i.e. when the waveform gets rendered=forged.

In order to keep all gates open, we can define a transformation, that does some operation on the symbols defined in the context.

In an equal manner as for the atom there should be a canon of transformations, so that they can easily be serialized and the whole sequence can be uniquely represented with a minimal necessary amount of data.

Such a transformation could be for example a solver for linear equations.
Here I define the transformation explicitly:

In [16]:
def my_transformation(in_dict):
    out_dict = copy(in_dict)
    out_dict['total_duration'] = out_dict['waiting_time'] + out_dict['pulse_time']
    return out_dict

The transformation gets associated with a `SegmentGroup` as in the following example. In principle they should be  also attachable to Segments, Elements and Sequences. This allows scoping of names, as the transformations get executed in the same order and can overwrite existing values.

In [ ]:
gs = SegmentGroup(s,s2, duration='total_duration', transformation=my_transformation)
out = gs.forge(100, waiting_time=1, pulse_time=1, phase=1)

Elements are straight forward. Here I attach the sequencing options to the Element. I think we should represent jump targets as references to element objects, as in a linked list, and do the derefenciation to indeces in the forging process of the sequence.

In [19]:
e = Element({'myChannel':gs}, {'nr_rep':2})
e.get_duration(waiting_time=1, pulse_time=1)

2

A Sequence is nothing more than a list of Elements. It can have sequencing information like an element and thus becomes a subsequence. (There are many ways in which we can implement subsequences, I don't have any preference yet, it could be a mixin: Sequenceable)

In [21]:
s = Sequence([e])
s.forge(10,{'waiting_time':1, 'pulse_time':2, 'phase': 1})

{0: {'sequencing': {'nr_rep': 2},
  'content': {1: {'data': {'myChannel': array([0.84147098, 0.89120736, 0.93203909, 0.96355819, 0.98544973,
            0.99749499, 0.9995736 , 0.99166481, 0.97384763, 0.94630009,
            0.84147098, 0.89120736, 0.93203909, 0.96355819, 0.98544973,
            0.99749499, 0.9995736 , 0.99166481, 0.97384763, 0.94630009,
            0.90929743, 0.86320937, 0.8084964 , 0.74570521, 0.67546318,
            0.59847214, 0.51550137, 0.42737988, 0.33498815, 0.23924933])}}},
  'type': 'element'}}

Forging works as expected.

A main part that needs to be implemented is the creation of a sequence from a list of parameter values. Say make a sequence from phase=1..10. But this should be easy.